# Flood from GEE

In [18]:
import ee
import geemap
import pandas as pd
import os

# Initialize Earth Engine
ee.Initialize(project="gee-ndvi-berhe")

# Define Area of Interest (South Sudan Bounding Box)
aoi = ee.Geometry.Rectangle([30.957, 7.480, 32.008, 9.205])

# Load Sentinel-1 SAR Dataset for Flood Mapping
sentinel1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(aoi)
    .filterDate("2015-01-01", "2024-12-31")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .select("VV")
)

# Compute Flooded Areas (Lower VV Values Indicate Water)
def flood_mask(img):
    water = img.lt(-18)  # Threshold for flood detection (adjustable)
    flood_area = water.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(), geometry=aoi, scale=30, maxPixels=1e13
    ).get("VV")
    
    # Ensure the flooded_area value is not None
    flood_area = ee.Algorithms.If(flood_area, flood_area, 0)
    
    return img.set("date", img.date().format()).set("flooded_area", flood_area)

# Apply Mask & Extract Time Series
flood_ts = sentinel1.map(flood_mask)

# Extract Dates & Flooded Areas
dates = flood_ts.aggregate_array("date").getInfo()
areas = flood_ts.aggregate_array("flooded_area").getInfo()

# Convert to DataFrame
output_folder = "flood_cover"
os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

df = pd.DataFrame({"date": dates, "flooded_area_m2_VV": areas})
df["date"] = pd.to_datetime(df["date"])

# Save the DataFrame
output_file = os.path.join(output_folder, "lankien_flood_timeseries_2015_2024.csv")
df.to_csv(output_file, index=False)

# Display Output
print(df.head())


                 date  flooded_area_m2
0 2015-01-11 03:41:25     1.449762e+06
1 2015-01-11 03:41:50     1.151573e+08
2 2015-01-11 03:42:15     4.799166e+06
3 2015-04-05 03:41:47     5.925780e+07
4 2015-04-05 03:42:16     6.412240e+05


Would you like visualizations? I can help you create time-series plots (e.g., line plots of flooded area over time).
Do you want further analysis? Trends, seasonality detection, or comparisons with climate variables could be interesting.

# Other data sources

In [31]:
import ee
import pandas as pd

# Initialize Google Earth Engine (GEE)
ee.Initialize(project="gee-ndvi-berhe")

# Define Lankien (South Sudan) bounding box
lankien = ee.Geometry.Rectangle([30.957, 7.480, 32.008, 9.205])

# Load Global Flood Database (GFD)
gfd = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1").filterBounds(lankien)

# Function to calculate flooded area for each image
def extract_flood_data(img):
    date = img.date().format("YYYY-MM-dd")  # Extract date
    flooded_pixels = img.select("flooded").gt(0).multiply(ee.Image.pixelArea())  # Convert to area (m²)
    flooded_area = flooded_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=lankien,
        scale=250,  # MODIS resolution is 250m
        maxPixels=1e13
    ).get("flooded")
    
    return ee.Feature(None, {"date": date, "flooded_area_m2": flooded_area})

# Apply the function over the collection
flood_ts = gfd.map(extract_flood_data)

# Convert to a FeatureCollection and extract results
flood_list = flood_ts.aggregate_array("date").getInfo()
flood_area_list = flood_ts.aggregate_array("flooded_area_m2").getInfo()

# Convert to a Pandas DataFrame
df = pd.DataFrame({"date": flood_list, "flooded_area_m2": flood_area_list})

# Convert to km²
df["flooded_area_km2"] = df["flooded_area_m2"] / 1e6

# Save to CSV
csv_filename = "lankien_flood_timeseries.csv"
df.to_csv(csv_filename, index=False)

# Display the first few rows
print(df.head())

print(f"✅ Flood time-series saved as: {csv_filename}")


         date  flooded_area_m2  flooded_area_km2
0  2001-08-06     5.532464e+05          0.553246
1  2002-04-26     0.000000e+00          0.000000
2  2006-08-12     0.000000e+00          0.000000
3  2007-07-03     2.253798e+07         22.537976
4  2007-07-29     6.021570e+06          6.021570
✅ Flood time-series saved as: lankien_flood_timeseries.csv


## Using Dynamic World in Google Earth Engine (GEE)
Dynamic World Land Cover Classes
The dataset classifies land cover into 10 categories:
Water
Trees
Grass
Crops
Shrub & Scrub
Built Area
Bare Ground
Snow & Ice
Flooded Vegetation
Cloud

In [63]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize(project="gee-ndvi-berhe")

# Load Dynamic World dataset
dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# Define Area of Interest (AOI)
aoi = ee.Geometry.Rectangle([31.003740, 7.813502, 33.000505, 9.301828])  # Example AOI (South Sudan)

# Filter by date and region
dw_filtered = dw.filterBounds(aoi).filterDate("2015-01-01", "2024-12-31")

# Select the most recent land cover classification
latest_dw = dw_filtered.sort("system:time_start", False).first()

# Visualize land cover
Map = geemap.Map()
Map.centerObject(aoi, 10)
Map.addLayer(latest_dw.select("label"), {
    "min": 0, "max": 9, "palette": [
        "419BDF", "397D49", "88B053", "E49635", "DFC35A",
        "C4281B", "A59B8F", "B39FE1", "397D49", "D0D0D0"
    ]}, "Land Cover Classification")
Map


Map(center=[8.55845984078217, 32.002122499999984], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
import pandas as pd
import os

def extract_land_cover(img):
    land_cover = img.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(), geometry=aoi, scale=30, maxPixels=1e13
    ).get("label")
    
    return img.set("date", img.date().format()).set("land_cover", land_cover)

# Apply function to extract land cover data
dw_ts = dw_filtered.map(extract_land_cover)

# Get dates & land cover histograms
dates = dw_ts.aggregate_array("date").getInfo()
land_cover_data = dw_ts.aggregate_array("land_cover").getInfo()

# Convert to DataFrame
df = pd.DataFrame({"date": dates, "land_cover_distribution": land_cover_data})
df["date"] = pd.to_datetime(df["date"])

# Save CSV
output_file = "land_cover_timeseries_2015_2024.csv"
df.to_csv(output_file, index=False)

print(df.head())
